In [1]:
import pandas as pd
import os
import sys

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
cwd = os.getcwd()

education_folder_path = os.path.dirname(cwd)

sys.path.append(education_folder_path)

In [3]:
from global_functions import remove_spaces_in_columns, up_load_shp, up_load_df, drop_geo, change_cols_names, get_newest_date_file, get_output_folder_location

In [4]:
#בתי ספר וגנים מעיריית בית שמש
BShemesh=up_load_df(r'{}\background_files\betshemesh_muni\חינוך'.format(education_folder_path),'מצבת מוסדות חינוך תשפג משרדי')
BShemesh=remove_spaces_in_columns(BShemesh)

In [5]:
BShemesh=BShemesh[~BShemesh['הערות'].str.contains('גן', na=False)]

In [6]:
# #קאורדינטות של מוסדות חינוך בכל הארץ ממשרד החינוך
# moe_mosdot_coordinates_2022=up_load_df(r'{}\background_files\ministry_of_education\מוסדות'.format(education_folder_path),'moe_mosdot_coordinates_2022')
# moe_mosdot_coordinates_2022=remove_spaces_in_columns(moe_mosdot_coordinates_2022)

#קאורדינטות של מוסדות חינוך בכל הארץ ממשרד החינוך
JTMT_setls_schools_coordinates=up_load_df(r'{}\background_files'.format(education_folder_path),'JTMT_setls_schools_coordinates')
JTMT_setls_schools_coordinates=remove_spaces_in_columns(JTMT_setls_schools_coordinates)

In [7]:
# moe_mosdot_coordinates_2022.rename(columns={'SEMEL_MOSAD': 'סמל_מוסד'}, inplace=True)
# moe_mosdot_coordinates_2022.rename(columns={'SHEM_MOSAD': 'שם מוסד'}, inplace=True)
# moe_mosdot_coordinates_2022.rename(columns={'ITM_X': 'coordinate_x'}, inplace=True)
# moe_mosdot_coordinates_2022.rename(columns={'ITM_Y': 'coordinate_y'}, inplace=True)

# moe_mosdot_coordinates_2022 = moe_mosdot_coordinates_2022.drop(columns=['UTM_X', 'UTM_Y', 'RAMAT_DIYUK_MIKUM'])

In [8]:
# מיזוג הטבלאות לפי העמודה "סמל_מוסד"
BShemesh = pd.merge(BShemesh, JTMT_setls_schools_coordinates[['סמל_מוסד', 'coordinate_x', 'coordinate_y']],
                     on='סמל_מוסד', how='left')

In [9]:
BShemesh=BShemesh.drop(columns=['שכונה', 'כתובת', 'טלפון', 'פקס', 'מייל', 'שם_מנהל/ת',
                                    'כתובת_מנהל', 'נייד', 'שם_מזכיר/ה', 'נייד.1', 'הערות', 'מפקח', 'קב"ט_מוס"ח',
                                    'גן_טרום_חובה', 'גן_חובה', 'גני_חינוך_מיוחד', 'סה"כ_גנים'])
BShemesh.head()

,מגזר,שם_המוסד,סמל_מוסד,כיתה_א1,כיתה_א2,כיתה_א3,כיתה_א4,כיתה_א5,כיתה_מקדמת,כיתה_ב1,כיתה_ב2,כיתה_ב3,כיתה_ב4,כיתה_מקדמת.1,כיתה_ג1,כיתה_ג2,כיתה_ג3,כיתה_ג4,כיתה_מקדמת.2,כיתה_ד1,כיתה_ד2,כיתה_ד3,כיתה_ד4,כיתה_מקדמת.3,כיתה_ה1,כיתה_ה2,כיתה_ה3,כיתה_ה4,כיתה_מקדמת.4,כיתה_ו1,כיתה_ו2,כיתה_ו3,כיתה_ו4,כיתה_מקדמת.5,"סה""כ_יסודי_מ""מ_וממ""ד",כיתה_ז1,כיתה_ז2,כיתה_ז3,כיתה_ז4,כיתה_מקדמת.6,כיתה_ח1,כיתה_ח2,כיתה_ח3,כיתה_ח4,כיתה_מקדמת.7,"סה""כ_יסודי_חרדי",כיתה_ט1,כיתה_ט2,כיתה_ט3,כיתה_מקדמת.8,כיתה_י1,כיתה_י2,כיתה_י3,כיתה_מקדמת.9,כיתה_יא1,כיתה_יא2,כיתה_יא3,כיתה_מקדמת.10,כיתה_יב1,כיתה_יב2,כיתה_יב3,כיתה_מקדמת.11,"_סה""כ_על_יסודי_מ""מ_וממ""ד","סה""כ_על_יסודי_חרדי",coordinate_x,coordinate_y
0,חרדי,אבי העזרי,227439.00,21.00,NaN,NaN,NaN,NaN,NaN,41.00,NaN,NaN,NaN,NaN,24.00,NaN,NaN,NaN,NaN,45.00,NaN,NaN,NaN,NaN,31.00,NaN,NaN,NaN,NaN,37.00,NaN,NaN,NaN,NaN,NaN,32.00,NaN,NaN,NaN,NaN,21.00,NaN,NaN,NaN,NaN,252.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,199464.00,624186.00
1,"ממ""ד",אהבת ישראל - בנות,246033.00,26.00,28.00,NaN,NaN,NaN,10.00,24.00,28.00,NaN,NaN,6.00,26.00,34.00,NaN,NaN,12.00,34.00,37.00,NaN,NaN,NaN,24.00,23.00,26.00,NaN,NaN,29.00,31.00,NaN,NaN,NaN,398.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,199373.00,624783.00
2,"ממ""ד",אהבת ישראל - בנים,194472.00,35.00,NaN,NaN,NaN,NaN,NaN,20.00,25.00,NaN,NaN,20.00,28.00,26.00,NaN,NaN,12.00,37.00,29.00,NaN,NaN,16.00,29.00,27.00,NaN,NaN,NaN,24.00,26.00,NaN,NaN,NaN,354.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,199373.00,624783.00
3,"ממ""ד",אולפנת אהבת ישראל,460667.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.00,40.00,NaN,NaN,13.00,32.00,35.00,NaN,NaN,12.00,NaN,38.00,41.00,NaN,NaN,27.00,27.00,NaN,NaN,31.00,NaN,NaN,9.00,22.00,25.00,NaN,8.00,393.00,NaN,199722.00,624804.00
4,"ממ""ד",אולפנת גילה - שעלי תורה,193185.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.00,30.00,NaN,NaN,5.00,29.00,25.00,NaN,NaN,10.00,NaN,41.00,36.00,35.00,NaN,30.00,34.00,NaN,6.00,28.00,27.00,26.00,11.00,16.00,19.00,23.00,NaN,461.00,NaN,199847.00,624128.00


In [10]:
# רשימת שמות השכבות (א' עד ו')
grades = ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'יא', 'יב']

# לולאה שמבצעת את הסכימה והמחיקה עבור כל שכבה
for i, grade in enumerate(grades):
    # איתור העמודות הרלוונטיות לכל שכבה (כולל כיתה מקדמת אם קיימת לאותה שכבה)
    grade_columns = [col for col in BShemesh.columns if col.startswith(f'כיתה_{grade}')]
    
    # הוספת העמודה של הכיתה המקדמת של אותה שכבה אם קיימת
    if f'כיתה_מקדמת.{i}' in BShemesh.columns:
        grade_columns.append(f'כיתה_מקדמת.{i}')
    elif 'כיתה_מקדמת' in BShemesh.columns and i == 0:
        grade_columns.append('כיתה_מקדמת')
    
    # סכימת העמודות אם יש כיתות רלוונטיות לשכבה הזו
    if grade_columns:
        BShemesh[f'כיתה_{grade}'] = BShemesh[grade_columns].sum(axis=1, skipna=True)
        
        # מחיקת העמודות שסוכמו
        BShemesh.drop(columns=grade_columns, inplace=True)

# institutions_table = pd.merge(institutions_table, BShemesh, on='סמל_מוסד', how='outer')
# institutions_table = institutions_table.dropna(axis=1, how='all')

BShemesh.head()

,מגזר,שם_המוסד,סמל_מוסד,"סה""כ_יסודי_מ""מ_וממ""ד","סה""כ_יסודי_חרדי","_סה""כ_על_יסודי_מ""מ_וממ""ד","סה""כ_על_יסודי_חרדי",coordinate_x,coordinate_y,כיתה_א,כיתה_ב,כיתה_ג,כיתה_ד,כיתה_ה,כיתה_ו,כיתה_ז,כיתה_ח,כיתה_ט,כיתה_י,כיתה_יא,כיתה_יב
0,חרדי,אבי העזרי,227439.00,NaN,252.00,NaN,NaN,199464.00,624186.00,21.00,41.00,24.00,45.00,31.00,37.00,32.00,21.00,0.00,0.00,0.00,0.00
1,"ממ""ד",אהבת ישראל - בנות,246033.00,398.00,NaN,NaN,NaN,199373.00,624783.00,64.00,58.00,72.00,71.00,73.00,60.00,0.00,0.00,0.00,0.00,0.00,0.00
2,"ממ""ד",אהבת ישראל - בנים,194472.00,354.00,NaN,NaN,NaN,199373.00,624783.00,35.00,65.00,66.00,82.00,56.00,50.00,0.00,0.00,0.00,0.00,0.00,0.00
3,"ממ""ד",אולפנת אהבת ישראל,460667.00,NaN,NaN,393.00,NaN,199722.00,624804.00,0.00,0.00,0.00,0.00,0.00,0.00,86.00,79.00,79.00,132.00,9.00,8.00
4,"ממ""ד",אולפנת גילה - שעלי תורה,193185.00,NaN,NaN,461.00,NaN,199847.00,624128.00,0.00,0.00,0.00,0.00,0.00,0.00,65.00,64.00,112.00,209.00,11.00,0.00


In [11]:
# יצירת עמודה "סה"כ" שמכילה את סכום כל העמודות הרלוונטיות
BShemesh['סה"כ_תלמידים'] = BShemesh[['סה"כ_יסודי_מ"מ_וממ"ד', 'סה"כ_יסודי_חרדי', '_סה"כ_על_יסודי_מ"מ_וממ"ד', 'סה"כ_על_יסודי_חרדי']].sum(axis=1)

# הסרת העמודות הישנות אם אינך זקוק להן יותר
BShemesh=BShemesh.drop(columns=['סה"כ_יסודי_מ"מ_וממ"ד', 'סה"כ_יסודי_חרדי', '_סה"כ_על_יסודי_מ"מ_וממ"ד', 'סה"כ_על_יסודי_חרדי'])

In [12]:
#sector
BShemesh.loc[BShemesh['מגזר']=='חרדי','sector'] = 2
BShemesh.loc[BShemesh['מגזר']=='ממ"ח','sector'] = 2
BShemesh.loc[BShemesh['מגזר']=='מ"מ','sector'] = 3
BShemesh.loc[BShemesh['מגזר']=='ממ"ד','sector'] = 3
BShemesh.loc[BShemesh['מגזר']=='עירוני','sector'] = 3

In [13]:
# סינון שורות שבהן העמודה "סה"כ" אינה שווה לאפס
BShemesh=BShemesh[BShemesh['סה"כ_תלמידים'] != 0]

In [14]:
# חלוקה לשכבות גיל
ages = {
    'ele': ['כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו'],
    'mid': ['כיתה_ז', 'כיתה_ח', 'כיתה_ט'],
    'high': ['כיתה_י', 'כיתה_יא', 'כיתה_יב']
}

# חישוב מספר התלמידים לכל שכבה
BShemesh['ele_stu'] = BShemesh[ages['ele']].sum(axis=1)
BShemesh['mid_stu'] = BShemesh[ages['mid']].sum(axis=1)
BShemesh['high_stu'] = BShemesh[ages['high']].sum(axis=1)

BShemesh_final = BShemesh.drop(['כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו', 'כיתה_ז', 'כיתה_ח', 'כיתה_ט', 'כיתה_י', 'כיתה_יא', 'כיתה_יב'], axis=1)

In [15]:
BShemesh.head()

,מגזר,שם_המוסד,סמל_מוסד,coordinate_x,coordinate_y,כיתה_א,כיתה_ב,כיתה_ג,כיתה_ד,כיתה_ה,כיתה_ו,כיתה_ז,כיתה_ח,כיתה_ט,כיתה_י,כיתה_יא,כיתה_יב,"סה""כ_תלמידים",sector,ele_stu,mid_stu,high_stu
0,חרדי,אבי העזרי,227439.00,199464.00,624186.00,21.00,41.00,24.00,45.00,31.00,37.00,32.00,21.00,0.00,0.00,0.00,0.00,252.00,2.00,199.00,53.00,0.00
1,"ממ""ד",אהבת ישראל - בנות,246033.00,199373.00,624783.00,64.00,58.00,72.00,71.00,73.00,60.00,0.00,0.00,0.00,0.00,0.00,0.00,398.00,3.00,398.00,0.00,0.00
2,"ממ""ד",אהבת ישראל - בנים,194472.00,199373.00,624783.00,35.00,65.00,66.00,82.00,56.00,50.00,0.00,0.00,0.00,0.00,0.00,0.00,354.00,3.00,354.00,0.00,0.00
3,"ממ""ד",אולפנת אהבת ישראל,460667.00,199722.00,624804.00,0.00,0.00,0.00,0.00,0.00,0.00,86.00,79.00,79.00,132.00,9.00,8.00,393.00,3.00,0.00,244.00,149.00
4,"ממ""ד",אולפנת גילה - שעלי תורה,193185.00,199847.00,624128.00,0.00,0.00,0.00,0.00,0.00,0.00,65.00,64.00,112.00,209.00,11.00,0.00,461.00,3.00,0.00,241.00,220.00


In [16]:
BShemesh_final.head()

,מגזר,שם_המוסד,סמל_מוסד,coordinate_x,coordinate_y,"סה""כ_תלמידים",sector,ele_stu,mid_stu,high_stu
0,חרדי,אבי העזרי,227439.00,199464.00,624186.00,252.00,2.00,199.00,53.00,0.00
1,"ממ""ד",אהבת ישראל - בנות,246033.00,199373.00,624783.00,398.00,3.00,398.00,0.00,0.00
2,"ממ""ד",אהבת ישראל - בנים,194472.00,199373.00,624783.00,354.00,3.00,354.00,0.00,0.00
3,"ממ""ד",אולפנת אהבת ישראל,460667.00,199722.00,624804.00,393.00,3.00,0.00,244.00,149.00
4,"ממ""ד",אולפנת גילה - שעלי תורה,193185.00,199847.00,624128.00,461.00,3.00,0.00,241.00,220.00


In [17]:
is_unique = BShemesh_final['סמל_מוסד'].is_unique
is_unique

False

In [18]:
# מציאת שורות עם ערכים כפולים בעמודה סמל_מוסד
duplicated_rows = BShemesh_final[BShemesh_final.duplicated(subset=['סמל_מוסד'], keep=False)]

In [19]:
# קיבוץ לפי "סמל_מוסד" עם סכימה של עמודות מסוימות והשארה של עמודות אחרות ללא שינוי
duplicated_rows = duplicated_rows.groupby('סמל_מוסד').agg({
    'מגזר': 'first',  # ערך ראשון
    'שם_המוסד': 'first',  # ערך ראשון
    'coordinate_x': 'first',  # ערך ראשון
    'coordinate_y': 'first',  # ערך ראשון
    'סה"כ_תלמידים': 'sum',  # ערך ראשון
    'sector': 'first',  # ערך ראשון
    'ele_stu': 'sum',    # סכימה
    'mid_stu': 'sum',    # סכימה
    'high_stu': 'sum',    # סכימה
}).reset_index()

duplicated_rows.head()

,סמל_מוסד,מגזר,שם_המוסד,coordinate_x,coordinate_y,"סה""כ_תלמידים",sector,ele_stu,mid_stu,high_stu
0,114397.00,חרדי,אור יעל,199497.00,627750.00,427.00,2.00,327.00,100.00,0.00
1,174748.00,חרדי,אור בית שמש - גור,199540.00,627386.00,861.00,2.00,636.00,225.00,0.00
2,238527.00,חרדי,ישיבת אור חדש,198583.00,628208.00,151.00,2.00,0.00,39.00,112.00
3,273086.00,חרדי,"בית יעקב רמב""ש",199476.00,625922.00,433.00,2.00,336.00,97.00,0.00
4,721175.00,חרדי,אולפנת שבות,198715.00,628209.00,200.00,2.00,0.00,57.00,143.00


In [20]:
BShemesh_final = pd.concat([duplicated_rows, BShemesh_final]).drop_duplicates(subset=['סמל_מוסד'], keep='first').reset_index(drop=True)

In [21]:
BShemesh_final = BShemesh_final.drop(['מגזר', 'סה"כ_תלמידים', 'שם_המוסד'], axis=1)

BShemesh_final = BShemesh_final.rename(columns={
    'סמל_מוסד': 'ID',
    'coordinate_x': 'x',
    'coordinate_y': 'y'
})

In [22]:
BShemesh_final.to_excel('BShemesh_final.xlsx', index=False)